In [1]:
print("Starting stuff")
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from secrets import *
import time
import random
import numpy
import pandas
import os
# print(secrets.passkey)
import json
import urllib.request

Starting stuff


In [2]:
driver = webdriver.Firefox()
time.sleep(2)
tag='artistsofinstagram'
driver.get('https://www.instagram.com/explore/tags/'+tag+'/')
time.sleep(2)

In [3]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [4]:
posts=[]
leads = soup.find_all("a")
print(len(leads))

58


## Find post links from hashtag page

In [5]:
for lead in leads:
    img=lead.find("img", class_="FFVAD")
    if(img):
        post={}
        post["url"]=lead.attrs['href']
        post["alt"]=img['alt']
        post["raw_link"]=img['srcset']
        post["tag"]=tag
        posts.append(post)
        
for post in posts:
    part=post["raw_link"].split(",")
    clean=part[-1].split()
    post["src"]=clean[0]

## Find details of post

In [6]:
for post in posts:
    url='https://www.instagram.com'+post["url"]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find("img", class_="FFVAD")
    try:
        post["raw_link"]=img['srcset']
        
        part=post["raw_link"].split(",")
        clean=part[-1].split()
        post["src"]=clean[0]
            
        post["size"]=clean[1]
         
    except:
        print("Some error")
    
    user = soup.find("h2").div.a
    post["user"]=user["href"]
    post["username"]=user.string
    
    raw_description=soup.find("h2").next_sibling
    post["plaintext_description"]=raw_description.get_text()
    post["rich_description"]=str(raw_description)
#     print(post["descriptionplain"], "\n \n")
#     print(post["rich_description"])
    
    

Some error


## Save images to /img/tag

In [7]:
directory= os.path.join(os.path.expanduser('~'),'Insta-hashtag', 'img', tag)
if not os.path.isdir(directory):
    os.makedirs(directory) 
    
for post in posts: 
    pic_url=post["src"]
    name=post["url"][3:-1]+'.jpg'
    path = os.path.join(directory,name)
    if (not os.path.isfile(path)):
        try:
            urllib.request.urlretrieve(pic_url, path)
            print("...",name )
        except:
            print("some error")
print("Completed")

... B_x1h0vF0kO.jpg
... B_wS-h9ptkC.jpg
... B_xdpOVjDh3.jpg
... B_yLczcDb-1.jpg
... B_xYXWcBmd1.jpg
... B_xRf0uDE4D.jpg
... B_xpeZBHNK0.jpg
... B_xfmelFfTl.jpg
... B_xhTTXDLgL.jpg
... B_yueabHZ18.jpg
... B_yudl5nVa-.jpg
... B_yua44JgzC.jpg
... B_yuauACMye.jpg
... B_yuaVmlut7.jpg
... B_yuZhIlguq.jpg
... B_yuYa5ATdz.jpg
... B_yuXKeFrMa.jpg
... B_yuWALFRHK.jpg
... B_yuU-3AoZX.jpg
... B_yuUhJjQbz.jpg
... B_yuUc9FxPW.jpg
... B_yuTLUFkVU.jpg
... B_yuOl1p34e.jpg
... B_yuMRJlDos.jpg
... B_yuMVgnfg5.jpg
... B_yuL1cDmBF.jpg
... B_yuLZchQBF.jpg
... B_yuKT5j1tF.jpg
... B_yuI7agjdB.jpg
... B_yuI8uHgry.jpg
... B_yuHEDJ8xi.jpg
... B_yuG7jhVC2.jpg
... B_yuGdCnKx_.jpg
Completed


In [12]:
with open('database.json','a') as db:
    db.write("var data=[ ")
    for post in posts:
        json.dump(post,db)
        db.write(", ")
    db.write(" ];")